# Pridobitev vektorskih predstavitev besedil

V tem zvezku predstavimo, kako lahko pridobimo vektorske predstavitve (vložitve) besed in dokumentov za analizo besedil.

Za začetek si preko API-ja pridobimo besedila zadnjih 100 predlogov vladi, ki vsebujejo vsaj 50 znakov (s tem pogojem poskrbimo, da po predprocesiranju ne dobimo praznega seznama pojavnic).

In [1]:
from textsemantics.server_api import ServerAPI

api = ServerAPI()
datasets = api.list_datasets()
metadata = api.get_metadata(datasets[2][0], sample_size=100, sampling_strategy='latest')

texts = api.get_texts(urls=metadata['text'])
texts = [text for text in texts if len(text) > 50]
print(f'Število predlogov vladi: {len(texts)}')

Število predlogov vladi: 99


Dobili smo 99 dokumentov. Zdaj lahko dokumente predprocesiramo tako, da iz njih odstranimo končnice, jih pretvorimo v seznam pojavnic, odstranimo prazne besede in lematiziramo preostale pojavnice.

In [2]:
import string
import nltk
nltk.download('stopwords', quiet=True)
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from lemmagen.lemmatizer import Lemmatizer
from lemmagen import DICTIONARY_SLOVENE
from IPython.display import display, Markdown

def preprocess(corpus):
    stop_words = set(stopwords.words('slovene'))
    tokenizer = RegexpTokenizer("\w+")
    lemmatizer = Lemmatizer(dictionary=DICTIONARY_SLOVENE)
    
    preprocessed = list()
    for text in corpus:
        text = text.translate(text.maketrans('', '', string.punctuation))
        tokens = tokenizer.tokenize(text.lower())
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words 
                  and len(token) > 2 and not token.isnumeric()]
        preprocessed.append(tokens)
        
    return preprocessed

tokens_list = preprocess(texts)

md_string = '### Prvih 10 pojavnic v prvem dokumentu\n'
for tok in tokens_list[0][:10]:
    md_string += f"- {tok}\n"
display(Markdown(md_string))

### Prvih 10 pojavnic v prvem dokumentu
- meniti
- slovenija
- obdavčiti
- gospodinjstvo
- podjetje
- obdavčiti
- verski
- institucija
- obdavčiti
- verski


Sedaj lahko vsak dokument predstavimo kot vektor. Vektorje bomo dobili z uporabo vreče besed, kjer vsak atribut predstavlja eno besedo v slovarju, vsaka vrstica pa en dokument. Tabela predstavi število pojavitev posamezne besede za posamezen dokument. Tabelo lahko prilagodimo tako, da upoštevamo pogostost besed - manj pogoste, a pomembne besede bodo imele višjo vrednost kot take, ki so vseprisotne. Poleg tega bomo dokumente predstavili z uporabo modela fastText, ki temelji na nevronskih mrežah in je prednaučen na velikem korpusu dokumentov. V osnovi je fastText učen, da besede predstavi z nizkodimenzionalnimi vektorji, vendar lahko vektorje dokumentov dobimo s povprečenjem vektorjev besed, ki se v dokumentu nahajajo.

In [3]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def vectorize(tokens_list, emb_type='fasttext'):
    joined_texts = [' '.join(tokens) for tokens in tokens_list]

    if emb_type=='fasttext':
        embedder = DocumentPoolEmbeddings([WordEmbeddings('sl')],
                                          pooling='mean')
        X = list()
        for i, doc in enumerate(joined_texts):
            sent = Sentence(doc)
            embedder.embed(sent)
            X.append(sent.embedding.cpu().detach().numpy())
        return np.array(X)
    elif emb_type == 'tfidf':
        return TfidfVectorizer().fit_transform(joined_texts)   
    return None

ft = vectorize(tokens_list, emb_type='fasttext')
tfidf = vectorize(tokens_list, emb_type='tfidf')

print(f'Matrika fastText: {ft.shape[0]} vrstic, {ft.shape[1]} stolpcev')
print(f'Matrika vreče besed (tf-idf): {tfidf.shape[0]} vrstic, {tfidf.shape[1]} stolpcev')

Matrika fastText: 99 vrstic, 300 stolpcev
Matrika vreče besed (tf-idf): 99 vrstic, 2270 stolpcev


Za vložitve tf-idf smo dobili 99 x 2270 matriko, za vložitve fastText pa 99 x 300 matriko. 

Dobljene vektorje si shranimo v datoteko, da bi jih lahko uporabljali v nadaljnjih primerih.

In [4]:
import os
from scipy.sparse import save_npz

def save_data(ft, tfidf):
    word_embs = list()
    embedder = WordEmbeddings('sl')

    for word in ['šola', 'počitnice', 'semafor', 'tehnologija']:
        sent = Sentence(word)
        embedder.embed(sent)
        vec = sent.tokens[0].embedding.cpu().detach().numpy()
        word_embs.append(vec)
    word_embs = np.array(word_embs)

    try:
        os.mkdir('data')
    except FileExistsError:
        pass
    np.save('data/ft.npy', ft)
    save_npz('data/tfidf.npz', tfidf)
    np.save('data/words.npy', word_embs)

save_data(ft, tfidf)